# langchain+qwen+qdrant实现RAG数据之间的隔离


我们在开发RAG增加检索应用时，总会遇到数据隔离的问题，比如，你上传的资料数据，不能让别人检索到，不然会存在数据安全问题。所以我们得为每个用户的数据进行隔离， 他们都不应该看到对方的数据，除非数据已经授权或者分配权限给他们访问。

本文主要实现该功能，只有进行了分配权限才能访问数据，该功能在企业或者RAG增强检索当中是比较常用的。

## 主要的实现步骤
不清楚RAG增强检索的可以看之前的文章
- 1.先加载文档
- 2.对文档数据进行分块
- 3.分块之后增加role权限字段，用于过滤权限
- 4.在检索文档数据时，会按role的值进行过滤
- 5.把过滤好后的文档数据，加上用户提的问题一起发给llm模型处理（通义千问）
- 6.LLM处理后，返回结果

该功能实现的核心是增加role 权限字段过滤，在用户提问的时候，就分配好了他能检索哪方面的数据。

# 一、安装相关依赖包

In [ ]:
!pip install --upgrade --quiet qdrant-client pypdfium2 backoff openai langchain_openai langchain langchain_community dashscope pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


# 二、导入相关依赖包

In [ ]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client.http import models as rest
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA

import os

qwen_api_key = "你的通义千问API-KEY"
os.environ["DASHSCOPE_API_KEY"] = qwen_api_key

# 三、构建向量化对象

In [ ]:
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=qwen_api_key
)

# 四、定义读取文档函数，并进行文档分块
读取文档数据的方法，并定义角色metadata.roles字段，分块后的文档都会有该字段，表示这个文档有哪些角色可以访问。

In [ ]:
def readPdfData(documents):
    page = []

    for document_path, roles in documents.items():
        # 读取当前目录下文件名为test.pdf和byd.pdf的文件
        pdf_loader = PyPDFLoader(document_path, extract_images=True)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )

        loaded_documents = pdf_loader.load_and_split()

        for doc in loaded_documents:
            doc.metadata["roles"] = roles

        # 对文档进行分块
        split_documents = text_splitter.split_documents(loaded_documents)
        page.extend(split_documents)
    return page

# 五、切换到Google Colab数据目录

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = "/content/drive/My Drive/Colab/RAG/LangChain_RAG/"
os.chdir(path)

读取llm文件，并设置角色过滤字段roles为ali，意思是检索文档数据时，参数中roles字段包含ali 的时候才能访问这个文档数据

In [ ]:
split_documents = readPdfData({"Data/llm.pdf": ["ali"]})

# 六、构建qdrant矢量数据库对象

存储数据的集合名称为my_documents，构建的方式是把文档数据加载到内存，仅用于测试，想构成存储到磁盘或者使用本地服务器的话参考官方提供的例子。

In [ ]:
# 内存方式
qdrant = Qdrant.from_documents(
    split_documents,
    embeddings,
    prefer_grpc=True,
    location=":memory:",  # 加载到内存
    collection_name="my_documents",
)

# 七、在之前的集合追加文档
读取maotai文件，并也为该文件分配了maotai权限

In [ ]:
# 该文档为茅台的财报，可多个pdf
documents = {
    "Data/maotai.pdf": ["maotai"],
}

split_documents = readPdfData(documents)

# 追加新的文档
qdrant.add_documents(split_documents, batch_size=20)

['b6f9950e4bc741a6a1921045193c8b62']

# 八、定义检索对象
检索文档数据库时，只会检索metadata.roles字段为maotai、ali的数据，其它数据都会过滤掉。

In [ ]:
user_roles = ["maotai", "ali"]

qdrant_retriever = qdrant.as_retriever(
    search_kwargs={
        "filter": rest.Filter(
            must=[
                rest.FieldCondition(
                    key="metadata.roles",
                    match=rest.MatchAny(any=user_roles)
                )
            ]
       )
    }
)

# 九、构建大语言模型对象

In [ ]:
llm = ChatTongyi(model="qwen-plus")

# 十、向qwen模型提问
根据用户提的问题，先去文档数据库中检索， 检索时会按我们定义的user_roles过滤掉数据，拿到相似的结果后，会根据用户的问题+相似的结果，一起发送给qwen模型，模型处理后会把最终的结果返回给我们。

In [ ]:
prompt_template = """
Question: {question}
使用源回答问题。如果没有答案，请说"文本中没有答案".

Source: {context}

### Response:
"""
# 构建提问模板
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# 构建过滤请求对象
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=qdrant_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# 发送请求
response = retrieval_qa.invoke({"query": "贵州茅台?"})
print(response["result"])

贵州茅台在2023年第三季度报告中展示了其主要财务数据和股东信息。以下是一些关键点：

1. **营业收⼊**: 为103,268,354,688.44元，同⽐增⻓了18.48%。
2. **净利润**: 归属于上市公司股东的净利润为52,876,217,064.12元，同⽐增⻓了19.09%。
3. **流动资产**:
   - 货币资⾦: 70,641,010,014.72元
   - 拆出资⾦: 95,625,606,731.69元
4. **负债和所有者权益总计**: 262,076,424,771.47元，其中所有者权益合计为225,018,799,759.41元。

此外，贵州茅台的前10名⽆限售条件股东中，中国贵州茅台酒⼚（集团）有限责任公司是最⼤股东，持有679,211,576股。其他重要股东包括⾹港中央结算有限公司和贵州省国有资本运营有限责任公司等。


还是提同样的问题，我们把该访问maotai.pdf的角色授权roles字段改为a，这里我们就不会访问到数据了，因为我们没有maotai权限，我们定义了只有这个权限才能访问该文件。

In [ ]:
# 改为a权限
user_roles = ["a", "ali"]

qdrant_retriever = qdrant.as_retriever(
    search_kwargs={
        "filter": rest.Filter(
            must=[
                rest.FieldCondition(
                    key="metadata.roles",
                    match=rest.MatchAny(any=user_roles)
                )
            ]
       )
    }
)

In [ ]:
prompt_template = """
Question: {question}
使用源回答问题。如果没有答案，请说"文本中没有答案".

Source: {context}

### Response:
"""
# 构建提问模板
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# 构建过滤请求对象
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=qdrant_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# 发送请求
response = retrieval_qa.invoke({"query": "贵州茅台?"})
print(response["result"])

文本中没有答案。
